In [4]:
import pandas as pd
import json

from sklearn.model_selection import train_test_split

In [5]:
system_sentiment = 'Sentiment analysis of input text and return 3 labels: "Pozitif", "Nötr", "Negatif"'

In [6]:
def format_training_sample(system_message, user_message, assistant_message):
    new_sample = {
                        "messages": [
                            {"role": "system", "content": system_message}, 
                            {"role": "user", "content": user_message}, 
                            {"role": "assistant", "content": assistant_message}
                        ]
                    }
    return new_sample

In [14]:
turkcell_df = pd.concat([pd.read_excel('../data/train_tweets.xlsx').dropna(), pd.read_excel('../data/test_tweets.xlsx').dropna()]).reset_index(drop=True)
turkcell_df

,Tweet,Label
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz
...,...,...
17284,@turktelekom kalite sıfır bi Turkcell değilsiniz,olumlu
17285,Turkcell ve Yandex Yerli Arama Motoru Üzerine ...,notr
17286,Turkcell yeni reklaminda altta kayan bilgilend...,notr
17287,Turkcell’de Yazılım Mühendisi maaşı hakkında b...,notr


In [16]:
turkcell_df['Label'].unique()

array(['olumsuz', 'olumlu', 'notr'], dtype=object)

In [17]:
turkcell_df["Label"].replace({"olumsuz": "Negatif", "olumlu": "Pozitif", "notr": "Nötr"}, inplace=True)
turkcell_df['Label'].unique()

array(['Negatif', 'Pozitif', 'Nötr'], dtype=object)

In [19]:
multiple_df = pd.concat([pd.read_csv('../data/multiple_sources_train.csv').dropna(), pd.read_csv('../data/multiple_sources_test.csv').dropna()]).reset_index(drop=True)
multiple_df

,text,label,dataset
0,ürünü hepsiburadadan alalı 3 hafta oldu. orjin...,Positive,urun_yorumlari
1,"ürünlerden çok memnunum, kesinlikle herkese ta...",Positive,urun_yorumlari
2,"hızlı kargo, temiz alışveriş.teşekkür ederim.",Positive,urun_yorumlari
3,Çünkü aranan tapınak bu bölgededir .,Notr,wiki
4,bu telefonu başlıca alma nedenlerim ise elimde...,Positive,urun_yorumlari
...,...,...,...
489639,Fransa bayrağı diğer kırmızı zeminden beyaz bi...,Notr,wiki
489640,Yine aynı yıl türkü dalında Murat Çobanoğlu il...,Notr,wiki
489641,Kurgunu skiyim oç evladı,Negative,tweet-pn
489642,Şarkı daha sonrasında Damian Marley tarafından...,Notr,wiki


In [24]:
multiple_df["label"].replace({"Negative": "Negatif", "Positive": "Pozitif", "Notr": "Nötr"}, inplace=True)
multiple_df['label'].unique()

C:\Users\assas\AppData\Local\Temp\ipykernel_19520\731228721.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  multiple_df["label"].replace({"Negative": "Negatif", "Positive": "Pozitif", "Notr": "Nötr"}, inplace=True)


array(['Pozitif', 'Nötr', 'Negatif'], dtype=object)

In [27]:
multiple_df.columns = ['Tweet', 'Label', 'Source']
multiple_df

,Tweet,Label,Source
0,ürünü hepsiburadadan alalı 3 hafta oldu. orjin...,Pozitif,urun_yorumlari
1,"ürünlerden çok memnunum, kesinlikle herkese ta...",Pozitif,urun_yorumlari
2,"hızlı kargo, temiz alışveriş.teşekkür ederim.",Pozitif,urun_yorumlari
3,Çünkü aranan tapınak bu bölgededir .,Nötr,wiki
4,bu telefonu başlıca alma nedenlerim ise elimde...,Pozitif,urun_yorumlari
...,...,...,...
489639,Fransa bayrağı diğer kırmızı zeminden beyaz bi...,Nötr,wiki
489640,Yine aynı yıl türkü dalında Murat Çobanoğlu il...,Nötr,wiki
489641,Kurgunu skiyim oç evladı,Negatif,tweet-pn
489642,Şarkı daha sonrasında Damian Marley tarafından...,Nötr,wiki


In [30]:
# Randomly select 100 tweets from each sentiment category in turkcell_df
turkcell_positive = turkcell_df[turkcell_df['Label'] == 'Pozitif'].sample(n=100, random_state=42)
turkcell_negative = turkcell_df[turkcell_df['Label'] == 'Negatif'].sample(n=100, random_state=42)
turkcell_neutral = turkcell_df[turkcell_df['Label'] == 'Nötr'].sample(n=100, random_state=42)

# Randomly select 1000 tweets from each sentiment category in multiple_df
multiple_positive = multiple_df[multiple_df['Label'] == 'Pozitif'].sample(n=1000, random_state=42)
multiple_negative = multiple_df[multiple_df['Label'] == 'Negatif'].sample(n=1000, random_state=42)
multiple_neutral = multiple_df[multiple_df['Label'] == 'Nötr'].sample(n=1000, random_state=42)

# Combine the selected tweets
combined_df = pd.concat([turkcell_positive, turkcell_negative, turkcell_neutral, multiple_positive, multiple_negative, multiple_neutral])

# Reset the index of the combined DataFrame
combined_df = combined_df.reset_index(drop=True).drop(columns=['Source'])

# Print the combined DataFrame
print(combined_df)

                                                  Tweet    Label
0     O işe turkcell bakıyomuş bak biz Ben o zaman a...  Pozitif
1     Turkcell'in şimdiye kadarki en iyi telefonu, h...  Pozitif
2      sağolsun turkcell mesaj atıyoda yalnız bırakmıyo  Pozitif
3     @ygtmerve @VodafoneTR en iyisi Turkcell canısı...  Pozitif
4     Bu sabah Turkcell'in mehmetcik reklamı beni bi...  Pozitif
...                                                 ...      ...
3295        Filmleri dünyanın birçok ülkesine satıldı .     Nötr
3296  Operasyonun ilk iki haftasında Musul civarında...     Nötr
3297                               İki kızları vardır .     Nötr
3298           Ortada sekizgen bir göbek taşı bulunur .     Nötr
3299      Olay sonucunda sekiz asker yaşamını yitirdi .     Nötr

[3300 rows x 2 columns]


In [32]:
X_train, X_test, y_train, y_test = train_test_split(combined_df["Tweet"], combined_df["Label"], test_size=0.1, stratify=combined_df["Label"])

In [36]:
train_json = []
for index, value in X_train.items():
 train_json.append(format_training_sample(system_sentiment, value, y_train[index]))
train_json

[{'messages': [{'role': 'system',
    'content': 'Sentiment analysis of input text and return 3 labels: "Pozitif", "Nötr", "Negatif"'},
   {'role': 'user',
    'content': 'bunlar düşünce filan da değil bunlar atmık bunlara saygı duyulmaz bunları sikip dışlayacaksın detroitli gibi çom'},
   {'role': 'assistant', 'content': 'Negatif'}]},
 {'messages': [{'role': 'system',
    'content': 'Sentiment analysis of input text and return 3 labels: "Pozitif", "Nötr", "Negatif"'},
   {'role': 'user', 'content': 'Filo olarak değiştirilmiştir .'},
   {'role': 'assistant', 'content': 'Nötr'}]},
 {'messages': [{'role': 'system',
    'content': 'Sentiment analysis of input text and return 3 labels: "Pozitif", "Nötr", "Negatif"'},
   {'role': 'user',
    'content': 'Aynı zamanda Reşat ve Şebnem adında iki çocuğu vardır .'},
   {'role': 'assistant', 'content': 'Nötr'}]},
 {'messages': [{'role': 'system',
    'content': 'Sentiment analysis of input text and return 3 labels: "Pozitif", "Nötr", "Negatif"'},

In [37]:
test_json = []
for index, value in X_test.items():
 test_json.append(format_training_sample(system_sentiment, value, y_test[index]))

In [38]:
with open('../jsonl/tweet-sentiment-train.jsonl', 'w', encoding='utf8') as jsonl_output:
    for entry in train_json:
        json.dump(entry, jsonl_output, ensure_ascii=False)
        jsonl_output.write('\n')

In [39]:
with open('../jsonl/tweet-sentiment-test.jsonl', 'w', encoding='utf8') as jsonl_output:
    for entry in test_json:
        json.dump(entry, jsonl_output, ensure_ascii=False)
        jsonl_output.write('\n')

In [44]:
pd.concat([X_train, y_train], axis=1).to_csv('../csv/tweet-sentiment-train.csv', index=False)
pd.concat([X_test, y_test], axis=1).to_csv('../csv/tweet-sentiment-test.csv', index=False)